<a href="https://colab.research.google.com/github/JasonYen-tw/CNN-Assignment-2025/blob/main/ACS111147_CNN_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ACS111147 - CNN 卷積神經網路實作 (CIFAR-10)

## 專案描述

這個專案的目標是在 CIFAR-10 數據集上訓練一個卷積神經網路（CNN）。CIFAR-10 包含 10 個類別的 32x32 彩色圖片，分別是：飛機、汽車、鳥、貓、鹿、狗、青蛙、馬、船和卡車。

## 實作重點
1.  **模型架構**：採用一個包含多個卷積層、LeakyReLU 激活函數、MaxPooling 和 Dropout 的深度 CNN 模型。
2.  **資料預處理**：將圖片像素值標準化到 [-0.5, 0.5] 區間，並對標籤進行 One-Hot 編碼。
3.  **資料增強 (Data Augmentation)**：使用 `ImageDataGenerator` 對訓練圖片進行即時的隨機旋轉、平移和水平翻轉，以提高模型的泛化能力。
4.  **模型訓練**：使用 Adamax 優化器和一個學習率排程器（Learning Rate Scheduler）來動態調整學習率，以達到更好的收斂效果。
5.  **自動化評分**：訓練完成後，模型會評估其在測試集上的表現，並將準確率、損失值等關鍵指標寫入 `model_accuracy.txt` 檔案中，以供 GitHub 自動化評分系統讀取。


In [ ]:
# -*- coding: utf-8 -*-
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
import numpy as np
import matplotlib.pyplot as plt

print("TensorFlow Version:", tf.__version__)
print("Keras Version:", keras.__version__)

In [ ]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

print("訓練資料維度:", x_train.shape, y_train.shape)
print("測試資料維度:", x_test.shape, y_test.shape)

NUM_CLASSES = 10
class_names = ["airplane", "automobile", "bird", "cat", "deer",
               "dog", "frog", "horse", "ship", "truck"]

cols = 8
rows = 2
plt.figure(figsize=(2 * cols, 2.5 * rows))
for i in range(cols * rows):
    random_index = np.random.randint(0, len(y_train))
    ax = plt.subplot(rows, cols, i + 1)
    ax.imshow(x_train[random_index, :])
    ax.set_title(class_names[y_train[random_index, 0]])
    ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
x_train_norm = (x_train / 255.0) - 0.5
x_test_norm = (x_test / 255.0) - 0.5

y_train_cat = to_categorical(y_train, NUM_CLASSES)
y_test_cat = to_categorical(y_test, NUM_CLASSES)

print("標準化後訓練圖片維度:", x_train_norm.shape)
print("One-Hot 編碼後訓練標籤維度:", y_train_cat.shape)

In [ ]:
def make_model():
    model = Sequential()
    model.add(Conv2D(16, (3, 3), input_shape=(32, 32, 3), padding='same'))
    model.add(LeakyReLU(0.1))
    model.add(Conv2D(32, (3, 3), padding='same'))
    model.add(LeakyReLU(0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3), padding='same'))
    model.add(LeakyReLU(0.1))
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(LeakyReLU(0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(LeakyReLU(0.1))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    return model

model = make_model()
model.summary()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
data_generator = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

INIT_LR = 5e-3
BATCH_SIZE = 32
EPOCHS = 25

model.compile(
    loss='categorical_crossentropy',
    optimizer=keras.optimizers.Adamax(learning_rate=INIT_LR),
    metrics=['accuracy']
)

def lr_scheduler(epoch):
    return INIT_LR * 0.9 ** epoch

In [ ]:
train_generator = data_generator.flow(x_train_norm, y_train_cat, BATCH_SIZE)
steps_per_epoch = len(x_train) // BATCH_SIZE

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS,
    callbacks=[LearningRateScheduler(lr_scheduler)],
    validation_data=(x_test_norm, y_test_cat),
    verbose=1
)

Epoch 1/25


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1288/1562 ━━━━━━━━━━━━━━━━━━━━ 34s 125ms/step - accuracy: 0.3672 - loss: 1.7205

In [ ]:
test_loss, test_acc = model.evaluate(x_test_norm, y_test_cat, verbose=2)

try:
    with open('model_accuracy.txt', 'w') as f:
        f.write("Model Performance Summary:\n")
        f.write(f"Test Accuracy: {test_acc:.4f}\n")
        f.write(f"Test Loss: {test_loss:.4f}\n")
        f.write(f"Final Training Accuracy: {history.history['accuracy'][-1]:.4f}\n")
        f.write(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}\n")
        f.write(f"Final Training Loss: {history.history['loss'][-1]:.4f}\n")
        f.write(f"Final Validation Loss: {history.history['val_loss'][-1]:.4f}\n")
        f.write(f"Training Epochs: {len(history.history['accuracy'])}\n")
        f.write(f"Model Parameters: {model.count_params()}\n")

    with open('model_accuracy.txt', 'r') as f:
        print("\n--- Content of model_accuracy.txt ---")
        print(f.read())
except Exception as e:
    print(f"Error writing to file: {e}")

In [ ]:
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.show()

In [ ]:
predictions = model.predict(x_test_norm)
predicted_labels = np.argmax(predictions, axis=1)

cols = 8
rows = 2
plt.figure(figsize=(2 * cols, 3 * rows))
for i in range(cols * rows):
    random_index = np.random.randint(0, len(y_test))
    ax = plt.subplot(rows, cols, i + 1)
    ax.imshow(x_test[random_index, :])
    pred_label = class_names[predicted_labels[random_index]]
    true_label = class_names[y_test[random_index, 0]]
    title_color = 'g' if pred_label == true_label else 'r'
    ax.set_title(f"Pred: {pred_label}\nTrue: {true_label}", color=title_color)
    ax.axis('off')
plt.tight_layout()
plt.show()